# Índice
1. [Configuração](#Configuração)
2. [Inspeção dos dados brutos](#Inspeção-dos-dados-brutos)
3. [Inspeção dos arquivos principais](#Inspeção-dos-arquivos-principais)
4. [Inspeção dos arquivos auxiliares](#Inspeção-dos-arquivos-auxiliares)

# Configuração

In [1]:
import pandas as pd
import sys
from IPython.core.display import HTML

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 300)
display(HTML("<style>.container { width:100% !important; } div.prompt {min-width: 10ex;} .rendered_html table { font-size: 11px; }</style>"))

## Inspeção dos dados brutos

Estas são as primeiras linhas do arquivo CSV referente às exportações:

In [2]:
exports_zip = '../data/exp.zip'
!zcat $exports_zip | head -n 4






gzip: stdout: Broken pipe


No caso das importações, as primeiras linhas do CSV são assim:

In [3]:
imports_zip = '../data/imp.zip'
!zcat $imports_zip | head -n 4






gzip: stdout: Broken pipe


## Inspeção dos arquivos principais

In [7]:
def read_and_group_file_in_chunks(file_name, verbose=False, chunksize=1000000):
    grouped_chunks = []
    if verbose:
        sys.stderr.write('Reading file in chunks of {} lines'.format(chunksize))
        sys.stderr.flush()
    # TODO: Load only the columns needed, with usecols=['foo', 'bar']
    for chunk in pd.read_csv(file_name, delimiter=";", chunksize=chunksize):
        if verbose:
            sys.stderr.write('.')
            sys.stderr.flush()
        grouped_chunks.append(chunk)
    if verbose:
        sys.stderr.write('\nConcatenating {} chunks\n'.format(
                         len(grouped_chunks)))
        sys.stderr.flush()
    return pd.concat(grouped_chunks)

In [8]:
df_exp = read_and_group_file_in_chunks(exports_zip, verbose=True)

Reading file in chunks of 1000000 lines.......................
Concatenating 23 chunks


In [9]:
df_exp.sample(5)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
366891,1997,4,84148019,11,63,SP,7,1010900,8,94,2032
7430615,2007,4,94036000,11,40,RS,1,920600,52,22256,100367
25079,1997,3,55161200,10,586,MG,9,910600,0,123,841
6940802,2006,11,39162000,10,586,SC,9,910600,0,4225,6141
7900288,2007,5,59069900,10,776,RS,4,817700,369,394,4329


In [10]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22648636 entries, 0 to 22648635
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   CO_NCM      int64 
 3   CO_UNID     int64 
 4   CO_PAIS     int64 
 5   SG_UF_NCM   object
 6   CO_VIA      int64 
 7   CO_URF      int64 
 8   QT_ESTAT    int64 
 9   KG_LIQUIDO  int64 
 10  VL_FOB      int64 
dtypes: int64(10), object(1)
memory usage: 1.9+ GB


In [11]:
def fix_dtypes(df):
    # Store categorical columns as such, reducing the memory usage
    # for the exports file from 1.9+ GB to 1.0 GB
    for col in df.columns[2:-3]:
        df[col]=df[col].astype('category')
    return df

In [13]:
df_exp = fix_dtypes(df_exp)
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22648636 entries, 0 to 22648635
Data columns (total 11 columns):
 #   Column      Dtype   
---  ------      -----   
 0   CO_ANO      int64   
 1   CO_MES      int64   
 2   CO_NCM      category
 3   CO_UNID     category
 4   CO_PAIS     category
 5   SG_UF_NCM   category
 6   CO_VIA      category
 7   CO_URF      category
 8   QT_ESTAT    int64   
 9   KG_LIQUIDO  int64   
 10  VL_FOB      int64   
dtypes: category(6), int64(5)
memory usage: 1.0 GB


In [14]:
df_exp.describe(include='category')

,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF
count,22648636,22648636,22648636,22648636,22648636,22648636
unique,11713,13,264,32,14,266
top,87089990,10,63,SP,1,817800
freq,134984,13534129,2091032,9216997,8614909,5022913


In [15]:
df_exp.describe(exclude='category')

,CO_ANO,CO_MES,QT_ESTAT,KG_LIQUIDO,VL_FOB
count,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07
mean,2.009973e+03,6.599953e+00,6.147536e+05,4.863214e+05,1.609096e+05
std,6.504889e+00,3.384929e+00,4.847814e+08,2.709977e+07,3.224087e+06
min,1.997000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.005000e+03,4.000000e+00,1.000000e+00,7.000000e+00,2.640000e+02
50%,2.010000e+03,7.000000e+00,2.300000e+01,1.120000e+02,2.292000e+03
75%,2.016000e+03,9.000000e+00,5.400000e+02,3.225000e+03,2.060300e+04
max,2.020000e+03,1.200000e+01,9.908988e+11,1.775019e+10,1.936700e+09


In [16]:
df_imp = read_and_group_file_in_chunks(imports_zip, verbose=True)

Reading file in chunks of 1000000 lines..................................
Concatenating 34 chunks


In [17]:
df_imp.sample(5)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
10333331,2007,2,42023100,11,160,DF,1,817800,10224,1178,2439
29150623,2018,2,39269069,10,160,RJ,4,817700,0,0,334
5957379,2003,4,84145990,11,161,RS,4,1017600,4,0,36
24261576,2015,8,39211390,10,160,AM,4,227700,4,4,234
3564720,2000,3,29159090,10,628,SP,4,817600,6,6,620


In [18]:
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783634 entries, 0 to 33783633
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   CO_NCM      int64 
 3   CO_UNID     int64 
 4   CO_PAIS     int64 
 5   SG_UF_NCM   object
 6   CO_VIA      int64 
 7   CO_URF      int64 
 8   QT_ESTAT    int64 
 9   KG_LIQUIDO  int64 
 10  VL_FOB      int64 
dtypes: int64(10), object(1)
memory usage: 2.8+ GB


In [19]:
df_imp = fix_dtypes(df_imp)
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783634 entries, 0 to 33783633
Data columns (total 11 columns):
 #   Column      Dtype   
---  ------      -----   
 0   CO_ANO      int64   
 1   CO_MES      int64   
 2   CO_NCM      category
 3   CO_UNID     category
 4   CO_PAIS     category
 5   SG_UF_NCM   category
 6   CO_VIA      category
 7   CO_URF      category
 8   QT_ESTAT    int64   
 9   KG_LIQUIDO  int64   
 10  VL_FOB      int64   
dtypes: category(6), int64(5)
memory usage: 1.5 GB


In [20]:
df_exp.describe(include='category')

,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF
count,22648636,22648636,22648636,22648636,22648636,22648636
unique,11713,13,264,32,14,266
top,87089990,10,63,SP,1,817800
freq,134984,13534129,2091032,9216997,8614909,5022913


In [21]:
df_exp.describe(exclude='category')

,CO_ANO,CO_MES,QT_ESTAT,KG_LIQUIDO,VL_FOB
count,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07
mean,2.009973e+03,6.599953e+00,6.147536e+05,4.863214e+05,1.609096e+05
std,6.504889e+00,3.384929e+00,4.847814e+08,2.709977e+07,3.224087e+06
min,1.997000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.005000e+03,4.000000e+00,1.000000e+00,7.000000e+00,2.640000e+02
50%,2.010000e+03,7.000000e+00,2.300000e+01,1.120000e+02,2.292000e+03
75%,2.016000e+03,9.000000e+00,5.400000e+02,3.225000e+03,2.060300e+04
max,2.020000e+03,1.200000e+01,9.908988e+11,1.775019e+10,1.936700e+09


## Inspeção dos arquivos auxiliares

In [ ]:
aux_file = '../data/aux.xlsx'
xls = pd.ExcelFile(aux_file)
print(xls.sheet_names)

In [ ]:
dfs = pd.read_excel(aux_file, sheet_name=None)
dfs['ÍNDICE']

In [ ]:
for i, sheet_name in enumerate(dfs):
    print('{}:\t{}'.format(sheet_name, dfs[sheet_name].shape))

In [ ]:
for sheet_name in dfs:
    print('== Sheet "{}" =='.format(sheet_name))
    display(dfs[sheet_name].sample(5))
    print(dfs[sheet_name].info())